In [84]:
import numpy as np

class ConvolutionLayer:
  # A Convolution layer

    # create filter matrix
    def createFilterMatrix(self):
        for i in range(self.num_filters):
            # initialize filter matrix with random
            self.weights[i] = np.random.normal(loc=0, scale=np.sqrt(1./(self.channel*self.width*self.height)), size=(self.channel, self.width, self.height))
        return 

    # initialization function to create convolutional layer instance
    def __init__(self, num_filters, inputs_channel, width, height, stride, padding, learning_rate):

        # number of convolutional filter as defined, it is required to be at least 1
        if num_filters <=0: 
             print("invalid filter number is provided.")
             return
        else:
            self.num_filters = num_filters
        # number of image channels. In the project, all image channel is decided to be 1 for gray image
        if inputs_channel <=0: 
             print("invalid channel number is provided.")
             return
        else:
            self.channel = inputs_channel
        # width of input images
        if width <=0: 
             print("invalid width of matrix is provided.")
             return
        else:
            self.width = width
        # height of input images
        if height <=0: 
             print("invalid height of matrix is provided.")
             return
        else:
            self.height = height
        # step size of convolutional matrix's sliding step
        if stride <=0: 
             print("Value of stride needs to be greater than 0 to be meaningful.")
             return
        else:
            self.stride = stride
        # padding for result
        if padding < 0: 
             print("padding can't be negative value.")
             return
        else:
            self.padding = padding
        # learning rate for gradient descent study
        self.lr = learning_rate

        # create random convolutional matrix
        self.weights = np.zeros((self.num_filters, self.channel, self.width, self.height))
        # bias matrix is set to all zero initially
        self.bias = np.zeros((self.num_filters,1))
        
        self.createFilterMatrix()

    
    def zero_padding(self, inputs, padding_size):
        # get input image's shape and calculate result image shape including padding size
        w, h = inputs.shape[0], inputs.shape[1]
        # new shape of matrix after considering padding
        new_w = 2 * padding_size + w
        new_h = 2 * padding_size + h
        out = np.zeros((new_w, new_h))
        # fill those zero spots by input values and rest will be left as it is
        if new_w > 0 and new_h > 0: 
            out[padding_size:w+padding_size, padding_size:h+padding_size] = inputs
        return out

    def calculateForwardMatrix(self, new_width, new_height):
        # corner check
        if new_width > 0 and new_height > 0:
            forward_matrix = np.zeros((self.num_filters, new_width, new_height))
            for f in range(self.num_filters):
                for w in range(new_width):
                    for h in range(new_height):
                        # sum up result to form forward matrix
                        forward_matrix[f,w,h]=np.sum(self.inputs[:,w:w+self.width,h:h+self.height]*self.weights[f,:,:,:])+self.bias[f]
            return forward_matrix
        else:
            print("either width or height is invalid")
            return

    def forward(self, inputs):
        # get input shape, height, width and channel
        channel = inputs.shape[0]
        width = inputs.shape[1]+2*self.padding
        height = inputs.shape[2]+2*self.padding
        self.inputs = np.zeros((channel, width, height))
        for ch in range(inputs.shape[0]):
            # process padding
            self.inputs[ch] = self.zero_padding(inputs[ch], self.padding)
        new_width = (width - self.width)//self.stride + 1
        new_height = (height - self.height)//self.stride + 1
        return self.calculateForwardMatrix(new_width, new_height)

    def backward(self, dy):

        dx = np.zeros(self.inputs.shape)
        dw = np.zeros(self.weights.shape)
        db = np.zeros(self.bias.shape)

        fil, width, height = dy.shape
        if fil > 0 and width > 0 and height > 0:
            for f in range(fil):
                for w in range(width):
                    for h in range(height):
                        dw[f]+=dy[f,w,h]*self.inputs[:,w:w+self.width,h:h+self.height]
                        dx[:,w:w+self.width,h:h+self.height]+=dy[f,w,h]*self.weights[f,:,:,:]
        else:
            return

        for f in range(fil):
            db[f] = np.sum(dy[f, :, :])

        self.weights -= self.lr * dw
        self.bias -= self.lr * db
        return dx
        

In [85]:
class ReLu:
    def forward(self, inputs):
        self.inputs = inputs
        relu = inputs.copy()
        # change the value that is less than 0 to 0
        relu[relu < 0] = 0
        return relu

    def backward(self, dy):
        dx = dy.copy()
        dx[self.inputs < 0] = 0
        return dx

In [86]:
class MaxPoolingLayer:
    def __init__(self, width, height, stride):
        if width > 0:
            self.width = width
        else:
            sys.exit("width cannot be negative") 
        if height > 0:
            self.height = height
        else:
            sys.exit("height cannot be negative") 
        if stride > 0:
            self.stride = stride
        else:
            sys.exit("stride cannot be negative") 

    def new_demension(self, current, old, stride):
        if current > 0 and old > 0 and stride > 0:
            return (current - old) // stride + 1
        else:
            return 0

    def forward_out(self, c_len, new_width, new_height):
        out = 0
        if c_len >= 0 and new_width >= 0 and new_height >= 0:
            out = np.zeros((c_len, new_width, new_height))
        else:
            sys.exit("Inputs should not be negative") 
        for c in range(c_len):
            for w in range(new_width):
                for h in range(new_height):
                    #np.max get the max value
                    out[c, w, h] = np.max(
                        self.inputs[c, w * self.stride:w * self.stride + self.width, h * self.stride:h * self.stride + self.height])
        return out

    def forward(self, inputs):
        if inputs is not None:
            self.inputs = inputs
        else:
            sys.exit("Inputs should not be None") 

        c_len, w_len, h_len = inputs.shape
        new_width = self.new_demension(w_len, self.width, self.stride)
        new_height = self.new_demension(h_len, self.height, self.stride)
        result = self.forward_out(c_len,new_width, new_height)
        if result is not None:
            return result
        else:
            sys.exit("error: output is none") 

    def backward_out(self, c_len, w_len, h_len, delta_y):
        dx = np.zeros(self.inputs.shape)
        for c in range(c_len):
            for w in range(0, w_len, self.width):
                for h in range(0, h_len, self.height):
                    st = np.argmax(self.inputs[c, w:w + self.width, h:h + self.height])
                    (idx, idy) = np.unravel_index(st, (self.width, self.height))
                    dx[c, w + idx, h + idy] = delta_y[c, w // self.width, h // self.height]
        return dx

    def backward(self, delta_y):
        c_len, w_len, h_len = self.inputs.shape
        if c_len < 0 or w_len < 0 or h_len < 0:
            sys.exit("Inputs should not be negative")
        result = self.backward_out(c_len, w_len, h_len, delta_y)
        return result

In [87]:
def cross_entropy(inputs, labels):
    return -np.log(np.sum(labels.reshape(1,labels.shape[0])*inputs))

In [88]:
class Flatten:
    def forward(self, i):
        self.c_len = i.shape[0]
        self.w_len = i.shape[1]
        self.h_len = i.shape[2]
        return i.reshape(1, i.shape[0]*i.shape[1]*i.shape[2])
    def backward(self, y):
        return y.reshape(self.c_len, self.w_len, self.h_len)
    def extract(self): 
        return

In [89]:
class FullyConnectedLayer:

    def __init__(self, num_inputs, num_outputs, learning_rate):
        if num_inputs < 0 or num_outputs < 0:
            print("invalid values for num_inputs and num_outputs")
            return
        self.weights = 0.01*np.random.rand(num_inputs, num_outputs)
        self.bias = np.zeros((num_outputs, 1))
        if learning_rate == 0:
            print("invalid values for learning_rate")
            return
        self.lr = learning_rate

    def forward(self, inputs):
        self.inputs = inputs
        return np.dot(self.inputs, self.weights) + self.bias.T

    def backward(self, dy):
        if dy.shape[0] == self.inputs.shape[0]:
            dy = dy.T
        dw = dy.dot(self.inputs)
        db = np.sum(dy, axis=1, keepdims=True)
        dx = np.dot(dy.T, self.weights.T)

        self.weights -= self.lr * dw.T
        self.bias -= self.lr * db

        return dx

In [90]:
class Softmax:
    def forward(self, inputs):
        tmp = np.exp(inputs, dtype=np.float)
        self.out = tmp/np.sum(tmp)
        return self.out
    def backward(self, dy):
        return self.out.T - dy.reshape(dy.shape[0],1)


In [91]:
import pickle
import sys


class CNN:
    def __init__(self):

        lr = 0.01
        self.layers = []
        self.layers.append(ConvolutionLayer(inputs_channel=1, num_filters=6, width=5, height=5, padding=2, stride=1, learning_rate=lr))
        self.layers.append(ReLu())
        self.layers.append(MaxPoolingLayer(width=2, height=2, stride=2))
        
        self.layers.append(ConvolutionLayer(inputs_channel=6, num_filters=16, width=5, height=5, padding=0, stride=1, learning_rate=lr))
        self.layers.append(ReLu())
        self.layers.append(MaxPoolingLayer(width=2, height=2, stride=2))
        
        self.layers.append(ConvolutionLayer(inputs_channel=16, num_filters=120, width=5, height=5, padding=0, stride=1, learning_rate=lr))
    
        self.layers.append(ReLu())
        self.layers.append(Flatten())
        self.layers.append(FullyConnectedLayer(num_inputs=120, num_outputs=60, learning_rate=lr))
        self.layers.append(ReLu())
        self.layers.append(FullyConnectedLayer(num_inputs=60, num_outputs=2, learning_rate=lr))
        self.layers.append(Softmax())
        self.lay_num = len(self.layers)


    def get_data(self, trn, i, batch):
        if batch > 0:
            self.b = batch
        else:
            sys.exit("batch cannot be negative") 
        if i + batch < trn.shape[0]:
            data = trn[i:i + batch]
            label = trn[i:i + batch]
        else:
            data = trn[i:trn.shape[0]]
            label = trn[i:trn.shape[0]]
        return data, label

    def forward(self, input_x):
        for l in range(self.lay_num):
            output = self.layers[l].forward(input_x)
            input_x = output
        return output

    def backward(self, output_y):
        for l in range(self.lay_num - 1, -1, -1):
            out = self.layers[l].backward(output_y)
            output_y = out

    def train(self, training_data, training_label, batch_size, epoch):
        total_acc = 0
        for e in range(epoch):
            for batch_index in range(0, training_data.shape[0], batch_size):
                # get the data
                data, label = self.get_data(training_data, i, batch_size) 
                loss = 0
                acc = 0

                for b in range(len(data)):
                    x = data[b]
                    y = label[b]

                    # forward
                    output = self.forward(x)

                    loss += cross_entropy(output, y)
                    if np.argmax(output) == np.argmax(y):
                        acc += 1
                        total_acc += 1

                    #backward
                    self.backward(y)

                # result
                loss /= batch_size
                batch_acc = float(acc) / float(batch_size)
                training_acc = float(total_acc) / float((batch_index + batch_size) * (e + 1))
 
    def predict(self, x):
        return self.forward(x)

    def test(self, data, label, test_size):
        total_acc = 0
        for i in range(test_size):
            x = data[i]
            y = label[i]

            predict_y = self.predict(x)
            if np.argmax(predict_y) == np.argmax(y):
                total_acc += 1
                
        return  float(total_acc) / float(test_size)

   

In [92]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 17 15:03:57 2020

@author: MoHan
"""

# libraries needed
import os

import math

import matplotlib.pyplot as plt
import numpy 

# note: if tensorflow is not install, run "pip install --upgrade tensorflow"
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image
import random

test_dir = "./dataset/test_set"
train_dir = "./dataset/training_set"

train_dir_cats = train_dir + "/cats"
train_dir_dogs = train_dir + "/dogs"
test_dir_cats = test_dir + "/cats"
test_dir_dogs = test_dir + "/dogs"

train_data = []
train_data_label = []
test_data = []
test_data_label = []

# Only transformed to gray pic
def normal_transform (imgpath):
    img = cv2.imread(imgpath)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (28,28))
    return Image.fromarray(img)

# Preprocessed using gaussian_canny
def gaussian_canny_transform (imgpath):
    img = cv2.imread(imgpath)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gaussian = cv2.GaussianBlur(img, (3,3), 0)
    gaussian = gaussian.astype(numpy.uint8)
    canny = cv2.Canny(gaussian, 50, 50)
    canny = cv2.resize(canny, (28,28))
    return Image.fromarray(canny)

# Reading training data
def read_training_data(train_data, train_data_label, dir, label):
    for filename in os.listdir(dir):
        imgpath = dir + "/" + filename
        img = normal_transform(imgpath)
        train_data.append([numpy.asarray(img)])
        train_data_label.append((label))

# Reading testing data
def read_testing_data(test_data, test_data_label, dir, label):
    for filename in os.listdir(dir):
        imgpath = dir + "/" + filename
        img = normal_transform(imgpath)
        test_data.append([numpy.asarray(img)])
        test_data_label.append((label))

#read gray images into train_data and train_data_label
read_training_data(train_data, train_data_label, train_dir_cats, [0,1])
#train_data =train_data[0:250]
#train_data_label =train_data_label[0:250]
read_training_data(train_data, train_data_label, train_dir_dogs, [1,0])
#train_data =train_data[0:500]
#train_data_label =train_data_label[0:500]
for i in range(0,len(train_data)//2,2):
               tmp=train_data[i]
               train_data[i] =train_data[len(train_data)-1-i]
               train_data[len(train_data)-1-i] =tmp
               #print(train_data_label[i])
               tlabel = train_data_label[i]
               train_data_label[i] = train_data_label[len(train_data)-1-i]
               train_data_label[len(train_data)-1-i] =  tlabel
               #print(train_data_label[i])
               



read_testing_data(test_data, test_data_label, test_dir_cats, [0,1])

read_testing_data(test_data, test_data_label, test_dir_dogs, [1,0])
for i in range(0,len(test_data)//2,3):
               tmp = test_data[i]
               test_data[i] =test_data[len(test_data)-1-i]
               test_data[len(test_data)-1-i] =tmp
               tlabel = test_data_label[i]
               test_data_label[i] = test_data_label[len(test_data)-1-i]
               test_data_label[len(test_data)-1-i] =  tlabel
               i=i+1
train_data = train_data[0:500]
train_data_label=train_data_label[0:500]
test_data=test_data[1000:1200]
test_data_label =test_data_label[1000:1200]

train_data = numpy.array(train_data)

test_data = numpy.array(test_data)

train_data_label = numpy.array(train_data_label)

test_data_label = numpy.array(test_data_label)



cnn = CNN()

#print('Training Lenet......')
cnn.train(training_data=train_data,training_label=train_data_label,batch_size=20,epoch=1)

#print('Testing Lenet......')
accuracy = cnn.test(data=test_data,label=test_data_label,test_size=200)
print('Testing accuracy:'+str(accuracy))


Testing accuracy:0.335


In [93]:
#Write result into txt
f = open("CNN_result.txt", "w")
f.write('Testing accuracy:'+str(accuracy*100)+'%')
f.close()

In [94]:
accuracy

0.335